<a href="https://colab.research.google.com/github/ysikalie/colorization/blob/master/coloriation_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as  tf
from keras.applications.densenet import DenseNet201
X = []
for filename in os.listdir('Train/'):
    X.append(img_to_array(load_img('Train/'+filename, target_size=(224, 224))))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X
X_Test = []
for filename in os.listdir('Test/'):
    X_Test.append(img_to_array(load_img('Test/'+filename,target_size=(224, 224))))
X_Test = np.array(X_Test, dtype=float)

Xtest = 1.0/255*X_Test

#Load weights
densenet = DenseNet201(weights='/home/zhaoyangze/PycharmProjects/ful_version/densenet201_weights_tf_dim_ordering_tf_kernels.h5' )


In [0]:
ensenet = Model(inputs=densenet.input, outputs=densenet.get_layer('pool2_conv').output)
densenet.graph = tf.get_default_graph()
densenet.summary()

In [0]:

#embed
embed_input = Input(shape=(56, 56,128))

#vgg block (56, 56 , 256)
# Block 1
gray_encoder_input = Input(shape=(224, 224, 1))
gray_encoder_output = Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1')(gray_encoder_input)
gray_encoder_output= Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2')(gray_encoder_output)
gray_encoder_output = MaxPooling2D((2, 2), strides=(2, 2), 
                                   name='block1_pool')(gray_encoder_output)

# Block 2
gray_encoder_output = Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1')(gray_encoder_output)
gray_encoder_output = Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2')(gray_encoder_output)


# Block 3
gray_encoder_output= Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1')(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3')(gray_encoder_output)
gray_encoder_output= MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(gray_encoder_output)

#Fusion
fusion_output = concatenate([gray_encoder_output, embed_input], axis=3)

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[gray_encoder_input,embed_input], outputs=decoder_output)
model.summary()


In [0]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (224, 224, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with densenet.graph.as_default(): 
        embed = densenet.predict(grayscaled_rgb_resized)
    return embed
# Image transformer
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True)

# Generate training data
batch_size = 1

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
#         grayscaled_rgb = gray2rgb(rgb2gray(batch))
#         embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        embed = create_inception_embedding(lab_batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
#         print([X_batch,lab_batch], Y_batch)
        yield ([X_batch, embed], Y_batch)


def test_image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtest, batch_size=batch_size): 
        rgb_2grayscaled = rgb2gray(batch)
        grayscaled_rgb = gray2rgb(rgb_2grayscaled)
        lab_batch = rgb2lab(grayscaled_rgb)
        embed = create_inception_embedding(lab_batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, embed], Y_batch)

model.compile(optimizer='rmsprop', loss='mse')
# model.summary()
model.fit_generator(image_a_b_gen(batch_size), epochs=2,
                    steps_per_epoch=1,
                    validation_data=test_image_a_b_gen(batch_size),
                    validation_steps=1)